<a href="https://colab.research.google.com/github/suleiman-odeh/NLP_Project_Team16/blob/main/Gemma_2/zero_shot_direct_Gemma_2_9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers bitsandbytes accelerate

In [ ]:
"""
This cell is for loading the model
I have run it already but cleared the output since it cant be upload to github
"""
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

# logging using user access token
login()

#  Define 4-Bit Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              # Loading in 4-bit
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, # Compute in 16-bit for speed, store in 4-bit
)

# Load model
model_id = "google/gemma-2-9b-it"

print(f"Loading {model_id} in 4-bit...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Model loaded successfully!")


In [ ]:
"""
This cell loads the dataset and keeps only the TEST split.
We only need test for zero-shot evaluation, but we keep the full df if you want quick checks.
"""

import pandas as pd

DATA_FILE = "QEvasion_cleaned.jsonl"

print(f"Loading data from {DATA_FILE} ...")
df = pd.read_json(DATA_FILE, lines=True)

test_df = df[df["split_type"] == "test"].copy()
print(f"Test rows: {len(test_df)}")

# Quick sanity check
print("\nColumns:", list(test_df.columns))
print(test_df[["question", "cleaned_answer", "clarity_id"]].head(2))